### Extract e mais algumas mudanças de transformação

In [3]:
!pip install pandasql pyspark

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=7e9e1dc61d671ad80debb1a550838cbd9507e99b693e363c2d66ece6af2484d7
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=1e9a551e9bacfff60b4e77f43281cd233629c525c74ddf9fe805f2609058325d
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pandasql pyspark


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as ps
import numpy as np
from pyspark.sql import SparkSession
import plotly.express as px

algora = pd.read_csv('algoraquant.csv')
algora['vl_valor'] = algora['vl_valor'].fillna(0)
algora['vl_vpa'] = algora['vl_vpa'].fillna(0)

import pandas as pd


# Função para calcular o novo valor
def calcular_valor_normal(row):
    # Limpa e formata 'vl_vpa' antes de converter para float
    vpa_str = str(row['vl_vpa']).replace('.', '').replace(',', '.').strip()  # Remove milhar e converte decimal
    try:
        vpa = float(vpa_str)  # Converte para float
    except ValueError:
        return None  # Retorna None se a conversão falhar

    # Remove o '%' e substitui a vírgula por ponto, depois converte para float
    variacao = float(row['vl_valor'].replace('%', '').replace(',', '.'))

    # Calcula o novo valor
    novo_valor = vpa * (1 + variacao / 100)
    return novo_valor

# Filtra as linhas onde 'vl_valor' contém '%'
filtro_percentual = algora['vl_valor'].str.contains('%', na=False)

# Aplica o cálculo apenas para essas linhas
algora.loc[filtro_percentual, 'vl_valor'] = algora.loc[filtro_percentual].apply(calcular_valor_normal, axis=1)

import numpy as np

def clean_patrimonio(value):
    if pd.isnull(value) or value == 'NaN':
        return 0  # Mantém como NaN para tratar depois

    value = str(value).replace('.', '').replace(',', '.')  # Normaliza ponto e vírgula
    if '%' in value:  # Se houver porcentagem, retorna 0
        return 0

    value = value.replace('$', '')  # Remove o cifrão se existir

    try:
        return float(value)  # Converte para float
    except ValueError:
        return np.nan  # Se não puder ser convertido, retorna NaN


# Aplicando a função na coluna 'vl_patrimonio'
algora['vl_patrimonio'] = algora['vl_patrimonio'].apply(clean_patrimonio)

# Verificando o resultado após a limpeza

# Calculando a média excluindo valores iguais a 0
mean_value_excluding_zero = algora[algora['vl_patrimonio'] != 0]['vl_patrimonio'].mean()

# Substituindo os valores 0 pela média calculada
algora['vl_patrimonio'] = algora['vl_patrimonio'].replace(0, mean_value_excluding_zero)

import pandas as pd

# Supondo que você já tem o DataFrame 'algora'

# Lista de colunas para as quais queremos aplicar a conversão
colunas_para_convertir = [
    'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis', 'vl_valor',
    'vl_liquidez_media_diaria', 'vl_pvp', 'vl_dividendo', 'vl_yeld',
    'vl_soma_yield_3m', 'vl_soma_yield_6m', 'vl_soma_yield_12m',
    'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m',
    'vl_soma_yield_ano_corrente', 'vl_variacao_cotacao_mes', 'vl_rentabilidade_mes',
    'vl_rentabilidade', 'vl_patrimonio', 'vl_vpa', 'vl_pvpa', 'vl_vpa_yield',
    'vl_vpa_change', 'vl_vpa_rent_m', 'vl_vpa_rent', 'vl_ativos',
    'vl_volatility', 'vl_numero_cotista', 'vl_vacancia_fisica', 'vl_vacancia_financeira'
]

# Função para converter a coluna para float
def converter_para_float(coluna):
    try:
        # Remove os pontos (separadores de milhar)
        coluna = coluna.str.replace('.', '', regex=False)
        # Substitui a vírgula por ponto (separador decimal)
        coluna = coluna.str.replace(',', '.', regex=False)
        # Converte para float
        return coluna.astype(float)
    except Exception as e:
        print(f"Erro ao converter coluna: {e}")
        return coluna  # Retorna a coluna original em caso de erro

# Aplica a conversão para cada coluna na lista
for coluna in colunas_para_convertir:
    if coluna in algora.columns:
        algora[coluna] = converter_para_float(algora[coluna])



Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: could not convert string to float: '1.19 %'
Erro ao converter coluna: could not convert string to float: '3.46 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.15 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.27 %'
Erro ao converter coluna: could not convert string to float: '2.48 %'
Erro ao converter coluna: could not convert string to float: '12.91 %'
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao co

Pronto estamos prontos para começar as nossas análises exploratórias, aqui deixamos apenas valores decimais facieís de serem tratados,
ou seja ... que não nenhum tipo de porcentagem envolvido, ficando cerca de 10 colunas de valores a serem análisados de acordo com a estatística

In [7]:
algora = algora.drop(columns=['Unnamed: 0','valor_decimal', 'vl_vpa_rent','vl_vpa_rent_m','vl_vpa_change','vl_vpa_yield','vl_pvpa','vl_rentabilidade',
                              'vl_rentabilidade_mes','vl_variacao_cotacao_mes','vl_soma_yield_ano_corrente','vl_yeld', 'vl_soma_yield_3m',
                              'vl_soma_yield_6m', 'vl_soma_yield_12m','vl_ativos', 'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m','vl_vacancia_financeira','vl_vacancia_fisica'])
# prenchendo os outros títulos com os valores de Humanidade

categoria_renomeacao = {
    'Shoppings': 'Tijolo Comercial',
    #'Lajes Corporativas': 'Tijolo Comercial',
    'Imóveis Comerciais - Outros': 'Tijolo Comercial',
    #'Agências de Bancos': 'Tijolo Comercial',
    'Varejo': 'Tijolo Comercial',
    'Imóveis Industriais e Logísticos': 'Tijolo Logístico/Industrial',
    'Logística': 'Tijolo Logístico/Industrial',
    'Imóveis Residenciais': 'Tijolo Comercial',
    'Residencial': 'Tijolo Comercial',
    'Papéis': 'Papel',
    'Títulos e Val. Mob.': 'Papel',
    'Títulos e Valores Mobiliários': 'Papel',
    'Fundo de Fundos': 'Híbrido',
    'Misto': 'Híbrido',
    'Híbrido': 'Híbrido',
    'Fundo de Desenvolvimento': 'Desenvolvimento',
    'Incorporações': 'Desenvolvimento',
    'Hotéis': 'Tijolo Comercial',
    #'Hotel': 'Tijolo Comercial',
    'Serviços Financeiros Diversos': 'Setor Específico',
    'Educacional': 'Setor Específico',
    'Hospitalar': 'Setor Específico',
    'Hospital': 'Setor Específico',
    'Fiagro': 'Setor Específico',
    'Agricultura': 'Setor Específico',
    'Indefinido': 'Indefinido',
    'Indefinido2016': 'Indefinido',
    'Indefinido2015': 'Indefinido',
    'Indefinido2017': 'Indefinido',
    'Renda': 'Renda'
}

# Aplicando a renomeação no DataFrame 'algora'
algora['nm_setor'] = algora['nm_setor'].replace(categoria_renomeacao)

numeric_columns = ['vl_media_taxa_homicidios', 'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis']

# Aplicamos o 'groupby' apenas nas colunas numéricas
algora[numeric_columns] = algora.groupby('nm_titulo')[numeric_columns].transform(lambda x: x.fillna(x.mean()))

algora['dt_inclusao'] = pd.to_datetime(algora['dt_inclusao'])

nm_titulo
0	CPTS11B
1	FIXX11
2	RBBV11
3	BZLI11
4	UBSR11
5	OUCY11
6	FOFT11
7	AFOF11
8	IFIE11
9	RBIV11
10	IFID11
11	QIFF11
12	EQIN11
13	ATSA11B
14	WPLZ11B
15	FIVN11
16	HMOC11
17	DMAC11
18	GRLV11
19	TRXL11
20	HGJH11
21	DOMC11
22	TBOF11
23	BBVJ11
24	BMLC11B
25	FFCI11
26	RDES11
27	FVBI11


Fundos que não existem ou pararam de existir ao longo dos anos ... 2016 até 202428	THRA11
29	GWIR11
30	AFCR11

### Distribuição de Dados Categóricas do Banco de Dados - FII - Correlação

In [8]:
import ipywidgets as widgets
import pandasql as ps
import pandas as pd


# Lista de categorias disponíveis
categorias = ['PAPEL', 'TIJOLO COMERCIAL', 'TIJOLO LOGíSTICO/INDUSTRIAL',
              'TIJOLO RESIDENCIAL',
              'HíBRIDO','DESENVOLVIMENTO', 'HOTéIS',
              'SETOR ESPECíFICO', 'INDEFINIDO', 'RENDA']


# Função que executa a query com base na seleção
def executar_query(categoria,dataframe):
    query = f"""
    SELECT *
    FROM algora
    WHERE
      dt_inclusao > '2021-08-15'
      AND UPPER(TRIM(nm_setor)) IN ('{categoria}')
    ORDER BY dt_inclusao DESC
    """

    # Executa a query usando pandasql
    gora = ps.sqldf(query, {"algora": dataframe})

    # Preenchendo valores nulos e ajustando a coluna 'vl_patrimonio'
    mean_value_excluding_zero = gora[gora['vl_patrimonio'] != 0]['vl_patrimonio'].mean()
    gora['vl_patrimonio'] = gora['vl_patrimonio'].fillna(0)
    gora['vl_patrimonio'] = gora['vl_patrimonio'].replace(0, mean_value_excluding_zero)

    # Exibe o número de linhas e colunas e descrição do DataFrame
    print(f"Temos {gora.shape[0]} linhas e {gora.shape[1]} colunas")

    # Obter a primeira e última data
    primeira_data = gora['dt_inclusao'].min()
    ultima_data = gora['dt_inclusao'].max()

    # Exibir os resultados
    print(f"A primeira data é: {primeira_data}")
    print(f"A última data é: {ultima_data}")

    return gora


# Executando a query inicial com o valor padrão
gora = executar_query(categorias[2], algora)

gora_numeric = gora.drop(['nm_titulo','nm_setor','dt_inclusao'], axis = 1)

import plotly.express as px

# Gera a matriz de correlação
correlation_matrix = gora_numeric.corr()

# Criando o gráfico de correlação com limites ajustados de -1 a 1
fig = px.imshow(correlation_matrix,
                text_auto=True,
                zmin=-1, zmax=1,  # Define os limites de -1 a 1
                color_continuous_scale='RdBu')  # Usa uma escala de cores adequada

fig.show()

Temos 427 linhas e 14 colunas
A primeira data é: 2021-10-01 00:00:00.000000
A última data é: 2024-09-01 00:00:00.000000


### Linha do Tempo Simplificada:

- **2016 - 2018**: Recuperação econômica lenta após recessão, juros elevados, FIIs em expansão inicial.
- **Junho de 2019**: Corte agressivo da Selic, cenário favorável para os FIIs.
- **2020 - 2021**: Pandemia de COVID-19 causa volatilidade, especialmente em FIIs de shoppings e escritórios.
- **2022 - 2024**: Retomada do mercado, elevação da Selic impacta o retorno dos FIIs, mas setores como logística e recebíveis ganham força.


### Modelo de Clusterização - Executar em Ambiente em Nuvem ou em PC Externo

In [ ]:
from pyspark.sql import SparkSession

# Cria a sessão Spark com configurações ajustadas

spark = SparkSession.builder\
  .appName('recomendador FII pySpark')\
  .getOrCreate()


In [ ]:
from pyspark.sql import functions as F

numeric_columns = [
    'vl_media_taxa_homicidios',
    'vl_avaliacao_media_ativos_fundo',
    'vl_quantidade_imoveis',
    'vl_valor',
    'vl_liquidez_media_diaria',
    'vl_pvp',
    'vl_dividendo',
    'vl_patrimonio',
    'vl_vpa',
    'vl_volatility',
    'vl_numero_cotista'
]

# Substituir os NaNs por 0 nas colunas numéricas
gora_spark = spark.createDataFrame(gora).na.fill(0, subset=numeric_columns)


aggregated_data = gora_spark.groupBy('nm_titulo').agg(
    F.mean('vl_media_taxa_homicidios').alias('mean_vl_media_taxa_homicidios'),
    F.mean('vl_avaliacao_media_ativos_fundo').alias('mean_vl_avaliacao_media_ativos_fundo'),
    F.mean('vl_quantidade_imoveis').alias('mean_vl_quantidade_imoveis'),
    F.mean('vl_valor').alias('mean_vl_valor'),
    F.mean('vl_liquidez_media_diaria').alias('mean_vl_liquidez_media_diaria'),
    F.mean('vl_pvp').alias('mean_vl_pvp'),
    F.mean('vl_dividendo').alias('mean_vl_dividendo'),
    F.mean('vl_patrimonio').alias('mean_vl_patrimonio'),
    F.mean('vl_vpa').alias('mean_vl_vpa'),
    F.mean('vl_volatility').alias('mean_vl_volatility'),
    F.mean('vl_numero_cotista').alias('mean_vl_numero_cotista')
)

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

def cria_pipeline_kmeans_cluster(aggregated_data):
    seed = 1224


    # Passo 2: Seleção das Features
    feature_columns = [
        'mean_vl_media_taxa_homicidios',
        'mean_vl_avaliacao_media_ativos_fundo',
        'mean_vl_quantidade_imoveis',
        'mean_vl_valor',
        'mean_vl_liquidez_media_diaria',
        'mean_vl_pvp',
        'mean_vl_dividendo',
        'mean_vl_patrimonio',
        'mean_vl_vpa',
        'mean_vl_volatility',
        'mean_vl_numero_cotista'
    ]

    assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

    # Passo 3: Escalonamento
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

    # Passo 4: PCA (opcional, pode ser útil para visualizar em 2D, mas pode não ser necessário para clustering)
    pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')

    # Passo 5: KMeans
    kmeans = KMeans(featuresCol='scaled_features', predictionCol='cluster').setK(7).setSeed(seed)

    # Construção do Pipeline
    pipeline = Pipeline(stages=[assembler, scaler,pca, kmeans])

    # Treinamento do Pipeline
    pipeline_model = pipeline.fit(aggregated_data)

    # Aplicação do Pipeline
    predictions = pipeline_model.transform(aggregated_data)

    # Extração das Features PCA para Visualização
    from pyspark.ml.functions import vector_to_array
    pca_feature_xy = predictions.withColumn('x', vector_to_array('pca_features')[0])\
                                .withColumn('y', vector_to_array('pca_features')[1])\
                                .select(['x', 'y', 'cluster', 'nm_titulo'])

    # Visualização com Plotly
    import plotly.express as px
    fig = px.scatter(pca_feature_xy.toPandas(), x='x', y='y', color='cluster',
                     hover_data=['nm_titulo'])
    fig.show()

    return pipeline_model

In [ ]:
pipeline_model = cria_pipeline_kmeans_cluster(aggregated_data)

### Realizando Trade de FII em um modo conservador com base em netraulidade = 0, vender = -1 e comprar = 1

Quero realizar um estudo em cima de uma estratégia de fundo quantitativo, por exemplo você recebe 100.000 mil reais, e a recomendação através de regressão trabalhará em cima do retorno de dividendos observando as variáveis que temos dentro de nosso Banco de Dados, e você passará para a gente o quanto de tempo que manterá aplicado e no final a aplicação de acordo com os tipos de trades que fez, retornará a quantia que você terá e quanto ganhou em cima das aplicações que o "robo" fez

Também mostraremos os trades em FII que fizemos ao longo de todos os meses, por exemplo a carteira que escolhemos ter: Foi de 5 ativos mais ou menos, comprando ,vendendo e mantendo, dependendo do divendo e riscos que poderíamos ter na aplicação até finalizar este 1 ano realizando os trades.

In [24]:
# Lista de colunas que você quer analisar
colunas_risco = ['vl_media_taxa_homicidios', 'vl_avaliacao_media_ativos_fundo',
                  'vl_liquidez_media_diaria','vl_pvp','vl_volatility']

# Verificar se todas as colunas existem no DataFrame
colunas_existentes = [col for col in colunas_risco if col in algora.columns]

# Aplicar o describe nas colunas filtradas
algora[colunas_existentes].describe()


,vl_media_taxa_homicidios,vl_avaliacao_media_ativos_fundo,vl_liquidez_media_diaria,vl_pvp,vl_volatility
count,8962.000000,8962.000000,5.883000e+03,2294.000000,445.000000
mean,17.105932,1.201680,4.400724e+05,8.033819,56.608360
std,28.679778,1.842617,1.388703e+06,106.980097,127.894745
min,0.000000,0.000000,0.000000e+00,0.010000,0.370000
25%,0.000000,0.000000,2.305000e+03,0.752500,11.720000
50%,0.000000,0.000000,2.838000e+04,0.930000,16.510000
75%,36.100000,3.335294,2.285800e+05,1.000000,38.350000
max,119.900000,5.000000,3.180000e+07,1961.670000,961.290000


In [9]:
import pandas as pd

# Ordenando os dados por ativo e data
gora['dt_inclusao'] = pd.to_datetime(gora['dt_inclusao'])
gora = gora.sort_values(by=['nm_titulo', 'dt_inclusao'])

# Verificar NaN e tratar
print(gora.isna().sum())
gora.fillna(method='ffill', inplace=True)  # Preenchendo com o valor anterior, por exemplo

nm_titulo                            0
vl_media_taxa_homicidios             0
vl_avaliacao_media_ativos_fundo      0
vl_quantidade_imoveis                0
nm_setor                             0
vl_valor                             2
vl_liquidez_media_diaria           111
vl_pvp                             178
vl_dividendo                         0
vl_patrimonio                        0
vl_vpa                               9
vl_volatility                      384
vl_numero_cotista                  384
dt_inclusao                          0
dtype: int64


<ipython-input-9-68c4f610567f>:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [10]:
# Criando lags para todas as variáveis de interesse
gora['vl_media_taxa_homicidios_lag1'] = gora.groupby('nm_titulo')['vl_media_taxa_homicidios'].shift(1)
gora['vl_avaliacao_media_ativos_fundo_lag1'] = gora.groupby('nm_titulo')['vl_avaliacao_media_ativos_fundo'].shift(1)
gora['vl_quantidade_imoveis_lag1'] = gora.groupby('nm_titulo')['vl_quantidade_imoveis'].shift(1)
gora['vl_valor_lag1'] = gora.groupby('nm_titulo')['vl_valor'].shift(1)
gora['vl_liquidez_media_diaria_lag1'] = gora.groupby('nm_titulo')['vl_liquidez_media_diaria'].shift(1)
gora['vl_pvp_lag1'] = gora.groupby('nm_titulo')['vl_pvp'].shift(1)
gora['vl_dividendo_lag1'] = gora.groupby('nm_titulo')['vl_dividendo'].shift(1)
gora['vl_patrimonio_lag1'] = gora.groupby('nm_titulo')['vl_patrimonio'].shift(1)
gora['vl_vpa_lag1'] = gora.groupby('nm_titulo')['vl_vpa'].shift(1)
gora['vl_volatility_lag1'] = gora.groupby('nm_titulo')['vl_volatility'].shift(1)
gora['vl_numero_cotista_lag1'] = gora.groupby('nm_titulo')['vl_numero_cotista'].shift(1)

# Remover as linhas com NaN gerados pelos lags
gora = gora.dropna()


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd

# Separando os dados em treino e teste
train = gora[gora['dt_inclusao'] < '2024-04-01']
test = gora[gora['dt_inclusao'] >= '2024-04-01']

# Features (variáveis explicativas) e Target (o que queremos prever)
X_train = train[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_train = train['vl_dividendo']

X_test = test[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_test = test['vl_dividendo']

# Criando e treinando o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Fazendo predições no conjunto de teste
y_pred = model.predict(X_test)

# Adicionando as predições ao DataFrame de teste para backtest
test['vl_dividendo_previsto'] = y_pred

# Calculando métricas de desempenho do modelo
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R²: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

# Exibindo as previsões e os valores reais para backtest
resultado = test[['dt_inclusao', 'nm_titulo', 'vl_dividendo', 'vl_dividendo_previsto']]
print("\nResultado do Backtest (Valores Reais x Previstos):")

resultado

R²: 0.9984361332153144
Mean Squared Error: 0.1347842597442166
Mean Absolute Error: 0.09735275106338717

Resultado do Backtest (Valores Reais x Previstos):


<ipython-input-11-0161accd5b14>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,dt_inclusao,nm_titulo,vl_dividendo,vl_dividendo_previsto
86,2024-04-01,AROA11,0.01,0.114542
44,2024-06-01,AROA11,0.01,0.114542
1,2024-09-01,AROA11,0.01,0.114542
87,2024-04-01,BLCP11,0.56,0.661731
45,2024-06-01,BLCP11,0.56,0.661731
...,...,...,...,...
83,2024-06-01,XPIN11,0.62,0.455667
41,2024-09-01,XPIN11,0.62,0.455667
126,2024-04-01,XPLG11,0.78,0.675176
84,2024-06-01,XPLG11,0.78,0.675176


In [44]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Função para executar trade com otimizações e observações
def executar_trade_optimizado_com_observacoes(mes, saldo_atual, portifolio, ativos_para_comprar):
    observacoes = []

    # Vender ativos com previsão de queda nos dividendos
    ativos_para_vender = portifolio.loc[portifolio['vl_dividendo_previsto'] < portifolio['vl_dividendo']]

    # Calculando saldo após venda
    valor_venda = ativos_para_vender['valor_investido'].sum()
    saldo_atual += valor_venda

    # Atualizando portifólio removendo os ativos vendidos e marcando trade = -1 (venda)
    portifolio.loc[ativos_para_vender.index, 'trade'] = -1
    observacoes.append(f"Vendidos {len(ativos_para_vender)} ativos devido à queda na previsão de dividendos.")
    portifolio = portifolio.drop(ativos_para_vender.index)

    # Selecionar ativos para compra (evitando duplicação e garantindo diversificação)
    ativos_comprados = set(portifolio['nm_titulo'].tolist())
    novas_linhas = []

    for ativo in ativos_para_comprar.itertuples():
        if ativo.nm_titulo not in ativos_comprados and saldo_atual > 0:
            investimento = min(saldo_atual / len(ativos_para_comprar), saldo_atual)
            novas_linhas.append([ativo.nm_titulo, investimento, ativo.vl_dividendo, ativo.vl_dividendo_previsto, 1])
            saldo_atual -= investimento
            ativos_comprados.add(ativo.nm_titulo)
            observacoes.append(f"Comprado ativo {ativo.nm_titulo} devido à boa previsão de dividendos e baixa volatilidade.")

    # Adicionar observação de manutenção
    if len(novas_linhas) == 0:
        observacoes.append("Nenhum novo ativo foi comprado, mantendo o portfólio atual.")

    # Verificar e garantir que há pelo menos 6 ativos no portfólio
    while len(ativos_comprados) < 6 and saldo_atual > 0:
        for ativo in ativos_para_comprar.itertuples():
            if ativo.nm_titulo not in ativos_comprados:
                investimento = min(saldo_atual / len(ativos_para_comprar), saldo_atual)
                novas_linhas.append([ativo.nm_titulo, investimento, ativo.vl_dividendo, ativo.vl_dividendo_previsto, 1])
                saldo_atual -= investimento
                ativos_comprados.add(ativo.nm_titulo)
                observacoes.append(f"Comprado ativo {ativo.nm_titulo} para garantir diversificação.")

            if len(ativos_comprados) >= 6:
                break

    # Adicionando novas linhas de uma só vez
    if novas_linhas:
        novas_linhas_df = pd.DataFrame(novas_linhas, columns=['nm_titulo', 'valor_investido', 'vl_dividendo', 'vl_dividendo_previsto', 'trade'])
        portifolio = pd.concat([portifolio, novas_linhas_df], ignore_index=True)

    # Atualizando ativos mantidos (sem alterações) para trade = 0
    portifolio.loc[portifolio['trade'].isna(), 'trade'] = 0

    return portifolio, saldo_atual, observacoes

# Função otimizada para rodar o processo de investimento com observações
def simular_investimento_optimizado_com_observacoes(saldo_inicial, meses_investimento):
    saldo = saldo_inicial
    portifolio = pd.DataFrame(columns=['nm_titulo', 'valor_investido', 'vl_dividendo', 'vl_dividendo_previsto', 'trade'])

    # Pré-selecionar ativos conservadores uma vez
    ativos_ordenados = test.sort_values(by='vl_volatility_lag1', ascending=True).head(20)

    for mes in pd.date_range('2024-10-01', periods=meses_investimento, freq='MS'):
        print(f"\nMês: {mes.strftime('%Y-%m')}")

        # Filtrar ativos disponíveis até a data do mês
        ativos_mes = ativos_ordenados[ativos_ordenados['dt_inclusao'] < mes]

        # Executar trade e obter observações
        portifolio, saldo, observacoes = executar_trade_optimizado_com_observacoes(mes, saldo, portifolio, ativos_mes)

        # Atualizando saldo com os dividendos recebidos
        dividendos_recebidos = (portifolio['vl_dividendo_previsto'] * portifolio['valor_investido']).sum()
        saldo += dividendos_recebidos

        print(f"Saldo Atual: R$ {saldo:.2f}")
        print(f"Portifólio Atual:\n{portifolio[['nm_titulo', 'valor_investido', 'trade']]}")

        # Exibir observações do mês
        print("Observações do mês:")
        for observacao in observacoes:
            print(f"- {observacao}")

    # Exibir saldo final após o período de investimento
    print(f"\nSaldo Final após {meses_investimento} meses: R$ {saldo:.2f}")
    return saldo

# Entrada de dados pelo usuário
saldo_inicial = float(input("Quanto você quer investir inicialmente (em reais)? "))
meses_investimento = int(input("Por quantos meses você deseja investir? "))

# Separar dados em treino e teste
train = gora[gora['dt_inclusao'] < '2024-04-01']
test = gora[gora['dt_inclusao'] >= '2024-04-01']

# Features (variáveis explicativas) e target (variável de saída)
X_train = train[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_train = train['vl_dividendo']

X_test = test[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_test = test['vl_dividendo']

# Criando e treinando o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Previsões
test['vl_dividendo_previsto'] = model.predict(X_test)

# Rodar simulação com otimizações e observações
saldo_final = simular_investimento_optimizado_com_observacoes(saldo_inicial, meses_investimento)

Quanto você quer investir inicialmente (em reais)? 500
Por quantos meses você deseja investir? 3

Mês: 2024-10
Saldo Atual: R$ 419.80
Portifólio Atual:
  nm_titulo  valor_investido trade
0    BLMG11        25.000000     1
1    GGRC11        23.750000     1
2    GLOG11        22.562500     1
3    BTSG11        21.434375     1
4    HLOG11        20.362656     1
5    GARE11        19.344523     1
6    LVBI11        18.377297     1
Observações do mês:
- Vendidos 0 ativos devido à queda na previsão de dividendos.
- Comprado ativo BLMG11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo GGRC11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo GLOG11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo BTSG11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo HLOG11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo GARE11 devido à boa previsão de dividendos e baixa volatili

<ipython-input-44-68e3f605f138>:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-44-68e3f605f138>:53: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

